In [1]:
import pandas as pd 
import numpy as np 
import anndata as an 
import sys, os 
sys.path.append('../src')

import sys
import os
import pathlib 
from typing import *

import torch
import numpy as np 
import pandas as pd 
import anndata as an
from functools import cached_property

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.preprocessing import LabelEncoder 

from models.lib.data import *
from models.lib.lightning_train import *
from models.lib.neural import *
from data.downloaders.external_download import *

from torchmetrics.functional import *

In [ ]:
class_label = 'cell_type'

module = DataModule(
    datafiles=['../data/dental/human_dental_T.h5ad'],
    labelfiles=['../data/dental/labels_human_dental.tsv'],
    class_label=class_label,
    sep='\t',
    datapath='../data/dental/',
    skip=3,
    batch_size=4,
    num_workers=0,
)

module.setup()

model = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    weights=total_class_weights(['../data/dental/labels_human_dental.tsv'], class_label, sep='\t'),
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau,
        'factor': 0.001,
    },
    metrics={
        'confusionmatrix': confusion_matrix,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    },
    weighted_metrics=False,
)

wandb_logger = WandbLogger(
    project=f"tabnet-classifer-sweep",
    name='Dental model local'
)

trainer = pl.Trainer(
    logger=wandb_logger
)

trainer.fit(model, datamodule=module)

Creating train/val/test DataLoaders...
Done, continuing to training.
Initializing network
Initializing explain matrix


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 1.4 M 
-----------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.770     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:219: UserWarning: You called `self.log('val_confusionmatrix', ...)` in your `validation_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# labels = pd.read_csv('../data/dental/labels_human_dental.tsv', sep='\t')
# labels = labels.sample(n=1000, weights=labels['cell_type'])

# labels.to_csv('../data/dental/labels_humans_dental_subset.tsv', sep='\t', index=False)
# pd.read_csv('../data/dental/labels_humans_dental_subset.tsv', sep='\t')['cell_type'].value_counts()